# Basic OpenQASM code

In [ ]:
from qdk import DepolarizingNoise
from qdk.openqasm import run, compile
from qdk.widgets import Histogram

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

Histogram(run(source, noise=DepolarizingNoise(0.01), as_bitstring=True))

In [ ]:
print(compile(source, as_bitstring=True))

## QASM and code

In [ ]:
from qdk import code
from qdk.openqasm import import_openqasm

source = """
    include "stdgates.inc";
    bit[2] c;
    qubit[2] q;
    h q[0];
    cx q[0], q[1];
    c = measure q;
"""

import_openqasm(source, name="bell")

code.bell()


In [ ]:
from qdk.openqasm import circuit
from qdk.widgets import Circuit

display(Circuit(circuit(code.bell)))

## Qiskit integration

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2, 2)
circuit.name = "state_prep"

# State vector to initialize: |ψ⟩ = (|0⟩ - |1⟩) / √2
circuit.initialize([1 / np.sqrt(2), -1 / np.sqrt(2)], 0)
circuit.h(0)
circuit.measure(0, 0)

circuit.prepare_state([1 / np.sqrt(2), -1 / np.sqrt(2)], 1)
circuit.h(1)
circuit.measure(1, 1)

circuit.draw(output="text")

In [ ]:
from qdk import TargetProfile
from qdk.qiskit import QSharpBackend

backend = QSharpBackend(target_profile=TargetProfile.Adaptive_RI)
job = backend.run(circuit)
counts = job.result().get_counts()
print(counts)

In [ ]:
print(backend.qir(circuit))

## Resource estimation


In [ ]:
from qdk.qsharp import init, eval, estimate
from qdk.widgets import EstimatesOverview

init()
with open("../estimation/EkeraHastadFactoring.qs", "r") as f:
    eval(f.read())

rsa_number = 1522605027922533360535618378132637429718068114961380688657908494580122963258952897654000350692006139

estimates = estimate(f"EkeraHastad({rsa_number.bit_length()}, {rsa_number}L, 7L)", [
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_ns_e3"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_ns_e4"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_us_e3"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_gate_us_e4"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_maj_ns_e4"}, "qecScheme": {"name": "floquet_code"}},
    {"errorBudget": 0.333, "qubitParams": {"name": "qubit_maj_ns_e6"}, "qecScheme": {"name": "floquet_code"}}
])

EstimatesOverview(
    estimates,
    colors=["#1f77b4", "#ff7f0e", "blue", "red", "green", "yellow"],
    runNames=["Gate ns e3, surface", "Gate ns e4, surface", "Gate us e3, surface", "Gate us e4, surface", "Majorana ns e4, floquet", "Majorana ns e6, floquet"]
)

## Azure submission

In [ ]:
from qdk import TargetProfile
from qdk.qsharp import init, eval, compile
from qdk.azure import Workspace

init(target_profile=TargetProfile.Base)

eval("operation Main() : Result { use q = Qubit(); H(q); return M(q);}")
operation = compile("Main()")

workspace = Workspace(
    subscription_id = "916dfd6d-030c-4bd9-b579-7bb6d1926e97",
    resource_group = "billti-team",
    name = "billti-quantum01",
    location = "westus2",
)

target = workspace.get_targets("quantinuum.sim.h2-2e")
print(target)
# job = target.submit(operation, "my-azure-quantum-job", shots=100)
# print(job.get_results())
